In [8]:
# Import packages
import osgeo
from os.path import join as pjoin
import pandas as pd
import numpy as np
import xarray as xr
import xrspatial as xrs
import rioxarray

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import pystac
from pystac_client import Client
import pystac.item_collection as pyic
import planetary_computer
import requests
import stackstac

import rich.table
import dask.diagnostics

scratch_dir = './Project/scratch' 

from pystac_client import Client
import planetary_computer as pc

In [2]:
# Define your area of interest
aoi_EMV = { #aoi Everest and Makalu Valley Polygon
  "type": "Polygon",
  "coordinates": [
    [
      [86.63380152490207, 28.56507505979343],
      [87.74001280776696, 28.479331576213163],
      [87.7789385926358, 28.213296130511523],
      [87.61479871540345, 27.906713498710847],
      [87.61479871540345, 27.906713498710847],
      [87.04816717665705, 27.239072299890207],
      [87.04816717665705, 27.239072299890207],
      [28.54835734379745, 86.06605505495027],
      [86.63380152490207, 28.56507505979343]
    ]
  ]
}

In [3]:
aoi_KV = { # aoi Kachenjunga Valley
"type": "Polygon",
  "coordinates": [
    [
    [86.63380152490207, 28.56507505979343],
    [87.74001280776696, 28.479331576213163],
    [87.7789385926358, 28.213296130511523],
    [87.61479871540345, 27.906713498710847],
    [87.61479871540345, 27.906713498710847],
    [87.15652306810863, 27.36778735019498],
    [89.06841422478996, 27.269554114998044],
    [89.06841422478996, 28.56507505979343],
    [86.63380152490207, 28.56507505979343]
    ]
  ]
}

In [89]:
daterange = [ 
    {"interval": ["1994-08-22T00:00:00Z", "1994-11-01T00:00:00Z"]},
    #{"interval": ["1999-08-22T00:00:00Z", "1999-11-01T00:00:00Z"]},
    {"interval": ["2004-08-22T00:00:00Z", "2004-11-01T00:00:00Z"]},
    #{"interval": ["2009-08-22T00:00:00Z", "2009-11-01T00:00:00Z"]},
    {"interval": ["2014-08-22T00:00:00Z", "2014-11-01T00:00:00Z"]},
    #{"interval": ["2019-08-22T00:00:00Z", "2019-11-01T00:00:00Z"]},
    {"interval": ["2022-08-22T00:00:00Z", "2022-11-01T00:00:00Z"]}]


In [91]:
# Define your temporal range
daterange1 = {"interval": ["1985-08-22T00:00:00Z", "1985-11-01T00:00:00Z"],
}
daterange2 = {"interval": ["1990-08-22T00:00:00Z", "1990-11-01T00:00:00Z"]}

In [92]:
catalog = Client.open(
  "https://planetarycomputer.microsoft.com/api/stac/v1"
)
# Define your search with CQL2 syntax
search = catalog.search(filter_lang="cql2-json", filter={
  "op": "and",
  "args": [
    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi_EMV]},
    {"op": "anyinteracts", "args": [{"property": "datetime"}, daterange1]},
    {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 5]}
  ]
})

first_item = next(search.get_items())
pc.sign_item(first_item).assets

charts = search.get_all_items()
print(len(charts))
print(charts)
#-------------------------------------------
for t in daterange:
    # Search against the Planetary Computer STAC API
    catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
        )
    
    # Define your search with CQL2 syntax
    search = catalog.search(filter_lang="cql2-json", filter={
        "op": "and",
        "args": [
            {"op": "s_intersects", "args": [{"property": "geometry"}, aoi_EMV]},
            {"op": "anyinteracts", "args": [{"property": "datetime"}, t]},
            {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
            {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 2]}
        ]
        }
    )

    first_item = next(search.get_items())
    pc.sign_item(first_item).assets

    items = search.get_all_items()
    print(len(items))
    charts = charts+items

#ds = stackstac.stack(PystacItemCollection(charts),epsg=32631)
print(len(charts))


9
70
64
114
118
375


In [96]:
ds = stackstac.stack(planetary_computer.sign(charts), epsg=6207)
ds

<xarray.DataArray 'stackstac-981d2eb3f6aca53fc65cb55c4fa676a5' (time: 375,
                                                                band: 22,
                                                                y: 154852,
                                                                x: 148873)>
dask.array<fetch_raster_window, shape=(375, 22, 154852, 148873), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/31)
  * time                         (time) datetime64[ns] 1985-08-27T06:26:40.41...
    id                           (time) <U31 'LT05_L2SP_163023_19850827_02_T1...
  * band                         (band) <U13 'qa' 'red' ... 'qa_aerosol'
  * x                            (x) float64 43.57 43.57 43.57 ... 90.0 90.0
  * y                            (y) float64 69.44 69.44 69.44 ... 27.76 27.76
    description                  (band) object 'Collection 2 Level-2 Quality ...
    ...                           ...
    title                        (band) object 'Surface Temperature Quality A...
    classification:bitfields     (band) object None ... [{'name': 'fill', 'le...
    common_name                  (band) object None None None ... 'coastal' None
    center_wavelength            (band) object None None None ... 10.9 0.44 None
    full_width_half_max          (band) object None None None ... 0.59 0.02 None
    epsg                         int32 6207
Attributes:
    spec:           RasterSpec(epsg=6207, bounds=(43.56952538501773, 27.76419...
    crs:            epsg:6207
    transform:      | 0.00, 0.00, 43.57|\n| 0.00,-0.00, 69.44|\n| 0.00, 0.00,...
    resolution_xy:  (0.0003118747432750979, 0.0002691502682949931)

In [98]:
with dask.diagnostics.ProgressBar():
    ts = ds.compute()

ts.rio.to_raster(pjoin(scratch_dir, 'ts.tif'), compress='LZW')